In [1]:
import pandas as pd
import requests
import tqdm
from datasets import load_dataset

url = "https://translate.apostroph.ch/api/unstable/v3/translate"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/117.0",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "de,en-US;q=0.7,en;q=0.3",
    "Accept-Encoding": "gzip, deflate, br",
    "Origin": "https://translate.apostroph.ch",
    "Referer": "https://translate.apostroph.ch/",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "TE": "trailers",
}
import time

def translate_text(text:str, source_language:str="en", target_language:str="pl"):
    
            data = {
                "segments": [{"text": text}],
                "source_language": source_language,
                "target_language": target_language,
                "politeness": "informal",
                "highlight_terms": False,
            }
            try:
                response = requests.post(url, json=data, headers=headers, timeout=10)
                response = response.json()["segments"][0]["text"]
                return response
            except:
                time.sleep(10)
                response = requests.post(url, json=data, headers=headers, timeout=10)
                print(response.json())
                response = response.json()["segments"][0]["text"]
                return response
                


c:\Users\SRU\.conda\envs\krakowiak\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# def check_2000_char_limit(conversations):
#     cur_chunk_len = 0
#     chunks = []
#     cur_left_index = 0
#     cur_right_index = 0

#     for i,conversation in enumerate(conversations):
#         cur_chunk_len += len(conversations[i]["content"])

#         if cur_chunk_len > 1850:
#             chunks.append((cur_left_index, i-1))
#             cur_left_index = i
#             cur_chunk_len = 0
#     chunks.append((cur_left_index,i))
#     return chunks

# def parse_chunks(chunk,conversation):
#     text_chunks = []
#     for idx in chunk:
#         concat_conversation = ""
#         for j in range(idx[0],idx[1]+1):
#             print(len(conversation[j]['content']))
#             concat_conversation += f"{conversation[j]['content']}<META_TAG>"
#             # print(concat_conversation)

#         text_chunks.append(concat_conversation)
#     return text_chunks
def check_2000_char_limit(conversations, max_length=2850):
    cur_chunk_len = 0
    chunks = []
    cur_left_index = 0

    for i, conversation in enumerate(conversations):
        message_length = len(conversation["content"])
        if cur_chunk_len + message_length > max_length and cur_chunk_len > 0:
            # Finish the current chunk before this message, as it would exceed the limit
            chunks.append((cur_left_index, i - 1))
            cur_left_index = i
            cur_chunk_len = message_length  # Start the next chunk with the current message
        else:
            # Add the current message's length to the current chunk
            cur_chunk_len += message_length

    # After processing all messages, add the remaining messages as the last chunk
    if cur_chunk_len > 0:
        chunks.append((cur_left_index, i))

    return chunks

def parse_chunks(chunk, conversation, max_chunk_length=2850):
    text_chunks = []
    for idx in chunk:
        concat_conversation = ""
        for j in range(idx[0], idx[1] + 1):
            content = conversation[j]['content']
            if len(concat_conversation) + len(content) <= max_chunk_length:
                concat_conversation += f"{content}<META_TAG>"
            else:
                remaining_chars = max_chunk_length - len(concat_conversation)
                concat_conversation += f"{content[:remaining_chars]}<META_TAG>"
                break

        text_chunks.append(concat_conversation)
    return text_chunks




In [3]:
ultra_zephyr = load_dataset("HuggingFaceH4/ultrachat_200k")


In [5]:
from datasets import load_dataset
zephyr_df = ultra_zephyr["train_sft"].to_pandas()
zephyr_df["messages_pl"] = zephyr_df["messages"]

for zephyr_df_idx,row in tqdm.tqdm(zephyr_df.iterrows(), total=len(zephyr_df)):
    msg = row["messages"]
    chunk =check_2000_char_limit(msg)
    concated_chunks = parse_chunks(chunk, msg)
    translations = [translate_text(j) for j in concated_chunks]
    concatenated_string = ' '.join(translations)
    translated_string = concatenated_string.split("<META_TAG>")
    for i, message in enumerate(msg):
        msg[i]["content"] = translated_string[i]

    zephyr_df.at[zephyr_df_idx,"messages_pl"] = msg
    if i%70000 == 0:
        zephyr_df.to_csv(f"../../data/pl/zephyr_pl_backup_{zephyr_df_idx}.csv")
zephyr_df.to_csv(f"../../data/pl/zephyr_pl.csv")


  0%|          | 6/207865 [00:44<425:01:05,  7.36s/it]


In [16]:
import pandas as pd

df = pd.read_csv("../../data/pl/zephyr_pl_backup_20000.csv")

In [20]:
df["messages_pl"].iloc[18000]

"[{'content': 'Stwórz kompleksowy przewodnik turystyczny po fikcyjnym miejscu docelowym, w tym informacje o najlepszych miejscach na pobyt, najlepszych lokalnych atrakcjach, obowiązkowych lokalnych potrawach i wskazówkach dotyczących poruszania się. Pamiętaj, aby dołączyć również urzekające obrazy i przyjąć żywy styl pisania, który angażuje czytelników i zachęca ich do odkrywania tego wyimaginowanego świata.', 'role': 'user'}\n {'content': ' Witamy w zaczarowanej krainie Xanadu, malowniczym raju położonym w sercu mistycznego lasu. Tutaj cuda ożywają, a turyści z całego świata uciekają przed jej niezwykłym pięknem. Xanadu oferuje magiczne doświadczenie nieporównywalne, ze starożytnymi ruinami, oszałamiającymi krajobrazami i przyjaznymi mieszkańcami. Ten przewodnik poprowadzi Cię przez najlepsze miejsca na pobyt, najważniejsze atrakcje, dania, które musisz spróbować i wskazówki dotyczące poruszania się po Xanadu.\\n\\nNajlepsze miejsca na pobyt\\nXanadu oferuje jedne z najbardziej luksus

In [19]:
## show the average length of the messages
lengths = []
df = df[:20000]
for i in range(len(df)):
    lengths.append(len(df["messages_pl"].iloc[i]))
sum(lengths)/len(lengths)

5528.13395